## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [1]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['/home/nirojasvar/random_sast/ExperimentationRSAST',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python310.zip',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/lib-dynload',
 '',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nirojasvar/random_sast/cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/cd_diagram']

In [2]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [3]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [4]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'SAST', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [5]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

directories=[]
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST')
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_All')
#directories.append(os.getcwd()+'/ResultsRsast/Server16_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/results_accuracy_per_ds')
#directories.append(os.getcwd()+'/results_accuracy_per_ds_10000')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


/tmp/ipykernel_71568/2832072796.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_71568/2832072796.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_71568/2832072796.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_71568/2832072796.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_71568/2832072796.py:29: FutureWarning: The default value of

In [6]:
df_used_sast=pd.read_excel("DataSetsUCLASummary.xlsx", dtype=str)
df_used_sast.columns

Index(['USED SAST', 'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'ID', 'Name', 'Type', 'Train ', 'Test ', 'Class', 'Length',
       'RSAST (10,10) WORST SAST?', 'O(|c|nm²)', 'O(|c|nm³)',
       'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 24'],
      dtype='object')

In [7]:
df_used_sast=df_used_sast[['Name', 'Type', 'Train ', 'Test ', 'Class', 'Length','USED SAST']]
df_used_sast=df_used_sast[df_used_sast['USED SAST']=='Y']
df_used_sast

,Name,Type,Train,Test,Class,Length,USED SAST
0,SmoothSubspace,Simulated,150,150,3,15,Y
1,Chinatown,Traffic,20,343,2,24,Y
2,ItalyPowerDemand,Sensor,67,1029,2,24,Y
3,MelbournePedestrian,Traffic,1194,2439,10,24,Y
4,Crop,Image,7200,16800,24,46,Y
...,...,...,...,...,...,...,...
77,BeetleFly,Image,20,20,2,512,Y
78,BirdChicken,Image,20,20,2,512,Y
79,Herring,Image,64,64,2,512,Y
80,Earthquakes,Sensor,322,139,2,512,Y


In [8]:
pv_all=pd.pivot_table(df_result,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
df_result=df_result[df_result.dataset_name.isin(ds_complete)]

In [9]:
#get ds tested overall
print("Total DS:" + str(len(df_result['dataset_name'].unique())))

Total DS:66


In [10]:
#get ds tested overall
df_result['dataset_name'].unique()

array(['MiddlePhalanxOutlineAgeGroup', 'FreezerSmallTrain', 'Wafer',
       'Earthquakes', 'Beef', 'DistalPhalanxOutlineAgeGroup', 'ECG200',
       'ShapeletSim', 'ItalyPowerDemand', 'ToeSegmentation2',
       'TwoLeadECG', 'ProximalPhalanxOutlineCorrect', 'CBF',
       'ToeSegmentation1', 'TwoPatterns', 'WordSynonyms', 'UMD',
       'Strawberry', 'FaceAll', 'GunPointMaleVersusFemale', 'PowerCons',
       'SonyAIBORobotSurface2', 'BirdChicken',
       'ProximalPhalanxOutlineAgeGroup', 'Herring', 'CricketY',
       'FiftyWords', 'FreezerRegularTrain', 'Trace', 'Wine',
       'MiddlePhalanxOutlineCorrect', 'BeetleFly', 'FaceFour',
       'ProximalPhalanxTW', 'SwedishLeaf', 'SonyAIBORobotSurface1',
       'PhalangesOutlinesCorrect', 'ECG5000', 'GunPoint', 'Meat',
       'SmoothSubspace', 'DiatomSizeReduction', 'CricketZ', 'CricketX',
       'ChlorineConcentration', 'ElectricDevices', 'Car',
       'GunPointAgeSpan', 'ECGFiveDays', 'Chinatown', 'Adiac',
       'DistalPhalanxOutlineCorrect'

In [11]:
#get ds tested overall
print("Total Param Combination:" + str(len(df_result['classifier_name'].unique())))

Total Param Combination:16


In [12]:
#get len methods of generated datasets
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [13]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [14]:
pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index()

classifier_name,dataset_name,ACF&PACF: n_random_points=10 nb_inst_per_class=1,ACF&PACF: n_random_points=10 nb_inst_per_class=10,ACF&PACF: n_random_points=30 nb_inst_per_class=1,ACF&PACF: n_random_points=30 nb_inst_per_class=10,Max PACF: n_random_points=10 nb_inst_per_class=1,Max PACF: n_random_points=10 nb_inst_per_class=10,Max PACF: n_random_points=30 nb_inst_per_class=1,Max PACF: n_random_points=30 nb_inst_per_class=10,None: n_random_points=10 nb_inst_per_class=1,None: n_random_points=10 nb_inst_per_class=10,None: n_random_points=30 nb_inst_per_class=1,None: n_random_points=30 nb_inst_per_class=10,all: n_random_points=10 nb_inst_per_class=1,all: n_random_points=10 nb_inst_per_class=10,all: n_random_points=30 nb_inst_per_class=1,all: n_random_points=30 nb_inst_per_class=10
0,Adiac,0.631714,0.576982,0.606138,0.569309,0.701790,0.497698,0.448082,0.502302,0.705882,0.623529,0.588235,0.639898,0.595396,0.549361,0.559079,0.547315
1,ArrowHead,0.748571,0.788571,0.745143,0.798857,0.761143,0.691429,0.665143,0.705143,0.697143,0.724571,0.629714,0.708571,0.728000,0.739429,0.723429,0.749714
2,BME,0.830667,0.832000,0.825333,0.829333,0.770667,0.802667,0.738667,0.805333,0.764000,0.865333,0.768000,0.857333,0.854667,0.868000,0.876000,0.873333
3,Beef,0.666667,0.680000,0.660000,0.693333,0.513333,0.640000,0.513333,0.646667,0.560000,0.700000,0.580000,0.726667,0.786667,0.833333,0.753333,0.833333
4,BeetleFly,0.780000,0.770000,0.800000,0.780000,0.640000,0.790000,0.620000,0.790000,0.540000,0.770000,0.680000,0.730000,0.770000,0.850000,0.740000,0.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,TwoPatterns,0.972950,0.995700,0.950200,0.996800,0.746150,0.961050,0.809150,0.849100,0.626300,0.973250,0.811800,0.887750,0.978250,0.996350,0.984050,0.996950
62,UMD,0.956944,0.973611,0.926389,0.977778,0.672222,0.833333,0.697222,0.847222,0.761111,0.941667,0.743056,0.945833,0.963889,0.975000,0.961111,0.973611
63,Wafer,0.998410,1.000000,0.997145,1.000000,0.979786,0.996269,0.990104,0.996690,0.975827,0.996755,0.985756,0.997502,0.996334,0.997404,0.997567,0.997567
64,Wine,0.851852,0.777778,0.781481,0.777778,0.688889,0.785185,0.703704,0.814815,0.611111,0.825926,0.607407,0.837037,0.818519,0.881481,0.840741,0.885185


In [15]:
"""
df_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how='right',right_on="dataset_name",left_on="Name").round(2)[['Name', 'ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=10','ACF&PACF: n_random_points=30 nb_inst_per_class=1','Type', 'Train ', 'Test ', 'Class', 'Length']]
df_tunned_rsast.sort_values("Name",inplace=True)
df_tunned_rsast.to_excel('ds_runned_rsast.xlsx')
"""

'\ndf_tunned_rsast=df_used_sast.merge(pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index(),how=\'right\',right_on="dataset_name",left_on="Name").round(2)[[\'Name\', \'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=10\',\'ACF&PACF: n_random_points=30 nb_inst_per_class=1\',\'Type\', \'Train \', \'Test \', \'Class\', \'Length\']]\ndf_tunned_rsast.sort_values("Name",inplace=True)\ndf_tunned_rsast.to_excel(\'ds_runned_rsast.xlsx\')\n'

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [16]:
filter_param=df_result

#filter_param=filter_param[filter_param.len_method=='ACF&PACF']
filter_param.dataset_name.unique()

array(['MiddlePhalanxOutlineAgeGroup', 'FreezerSmallTrain', 'Wafer',
       'Earthquakes', 'Beef', 'DistalPhalanxOutlineAgeGroup', 'ECG200',
       'ShapeletSim', 'ItalyPowerDemand', 'ToeSegmentation2',
       'TwoLeadECG', 'ProximalPhalanxOutlineCorrect', 'CBF',
       'ToeSegmentation1', 'TwoPatterns', 'WordSynonyms', 'UMD',
       'Strawberry', 'FaceAll', 'GunPointMaleVersusFemale', 'PowerCons',
       'SonyAIBORobotSurface2', 'BirdChicken',
       'ProximalPhalanxOutlineAgeGroup', 'Herring', 'CricketY',
       'FiftyWords', 'FreezerRegularTrain', 'Trace', 'Wine',
       'MiddlePhalanxOutlineCorrect', 'BeetleFly', 'FaceFour',
       'ProximalPhalanxTW', 'SwedishLeaf', 'SonyAIBORobotSurface1',
       'PhalangesOutlinesCorrect', 'ECG5000', 'GunPoint', 'Meat',
       'SmoothSubspace', 'DiatomSizeReduction', 'CricketZ', 'CricketX',
       'ChlorineConcentration', 'ElectricDevices', 'Car',
       'GunPointAgeSpan', 'ECGFiveDays', 'Chinatown', 'Adiac',
       'DistalPhalanxOutlineCorrect'

In [17]:
filter_param.rpoint.unique()

array(['10', '30'], dtype=object)

In [18]:
#filter_param=filter_param[filter_param.rpoint=='half_len']

In [19]:
filter_param.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [20]:
#filter_param=filter_param[filter_param.nb_per_class=='30']

In [21]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="SAST"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df


,dataset_name,ACF&PACF,Max PACF,None,all,score,method
0,Adiac,0.596036,0.537468,0.639386,0.562788,0.68,SAST
1,ArrowHead,0.770286,0.705714,0.690000,0.735143,0.77,SAST
2,BME,0.829333,0.779333,0.813667,0.868000,0.87,SAST
3,Beef,0.675000,0.578333,0.641667,0.801667,0.80,SAST
4,BeetleFly,0.782500,0.710000,0.680000,0.802500,0.80,SAST
...,...,...,...,...,...,...,...
61,TwoPatterns,0.978913,0.841363,0.824775,0.988900,0.99,SAST
62,UMD,0.958681,0.762500,0.847917,0.968403,0.98,SAST
63,Wafer,0.998889,0.990712,0.988960,0.997218,1.00,SAST
64,Wine,0.797222,0.748148,0.720370,0.856481,0.85,SAST


In [22]:
merged_df.describe()

,ACF&PACF,Max PACF,None,all,score
count,66.000000,66.000000,66.000000,66.000000,66.000000
mean,0.832099,0.791342,0.790074,0.833102,0.846212
std,0.126609,0.132509,0.124594,0.136696,0.123423
min,0.530844,0.537468,0.537662,0.441830,0.530000
25%,0.755632,0.705824,0.688077,0.736955,0.760000
50%,0.851630,0.795498,0.801316,0.858441,0.860000
75%,0.946078,0.929473,0.902334,0.957732,0.960000
max,1.000000,0.990712,0.989048,1.000000,1.000000


In [23]:

# Summary statistics for a Series (single variable)

summ_ds=filter_param
summ_ds=researchpy.summary_cont(summ_ds.groupby(['dataset_name'])['accuracy'], conf = 0.95)
summ_ds

,N,Mean,SD,SE,95% Conf.,Interval
dataset_name,,,,,,
Adiac,80,0.5839,0.0683,0.0076,0.5687,0.5991
ArrowHead,80,0.7253,0.0497,0.0056,0.7142,0.7364
BME,80,0.8226,0.0545,0.0061,0.8104,0.8347
Beef,80,0.6742,0.1106,0.0124,0.6496,0.6988
BeetleFly,80,0.7438,0.1101,0.0123,0.7193,0.7682
...,...,...,...,...,...,...
TwoPatterns,80,0.9085,0.1133,0.0127,0.8833,0.9337
UMD,80,0.8844,0.1140,0.0128,0.8590,0.9098
Wafer,80,0.9939,0.0076,0.0008,0.9923,0.9956


In [24]:
stats = filter_param.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF&PACF    0.832099   1320  0.131866  0.839213  0.824985
Max PACF    0.791342   1320  0.144522  0.799139  0.783546
None        0.790074   1320  0.141438  0.797704  0.782444
all         0.833102   1320  0.138908  0.840596  0.825609


#### Generate Boxplot Tunning Hyperparameter

In [25]:
df_result.nb_per_class.unique()

array(['1', '10'], dtype=object)

In [26]:
df_result.len_method.unique()

array(['ACF&PACF', 'Max PACF', 'None', 'all'], dtype=object)

In [27]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):

    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.accuracy)
        min_bx=min(df_to_cd.accuracy)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_acc/boxplot_acc'+len_m+ints+'.png')




In [28]:
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [29]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
            print("ds:"+ds)
            print("ints:"+str(ints))
            print("len_m:"+len_m)

            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.accuracy)
            min_bx=min(df_to_cd.accuracy)
            
            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]    

            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            
            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_acc_per_ds/boxplot_acc'+len_m+ints+'_'+ds+'.png')

ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:ACF&PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:Max PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:None
ds:MiddlePhalanxOutlineAgeGroup
ints:1
len_m:all
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:ACF&PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:Max PACF
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:None
ds:MiddlePhalanxOutlineAgeGroup
ints:10
len_m:all
ds:FreezerSmallTrain
ints:1
len_m:ACF&PACF
ds:FreezerSmallTrain
ints:1
len_m:Max PACF
ds:FreezerSmallTrain
ints:1
len_m:None
ds:FreezerSmallTrain
ints:1
len_m:all
ds:FreezerSmallTrain
ints:10
len_m:ACF&PACF


/tmp/ipykernel_71568/3028681140.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


ds:FreezerSmallTrain
ints:10
len_m:Max PACF
ds:FreezerSmallTrain
ints:10
len_m:None
ds:FreezerSmallTrain
ints:10
len_m:all
ds:Wafer
ints:1
len_m:ACF&PACF
ds:Wafer
ints:1
len_m:Max PACF
ds:Wafer
ints:1
len_m:None
ds:Wafer
ints:1
len_m:all
ds:Wafer
ints:10
len_m:ACF&PACF
ds:Wafer
ints:10
len_m:Max PACF
ds:Wafer
ints:10
len_m:None
ds:Wafer
ints:10
len_m:all
ds:Earthquakes
ints:1
len_m:ACF&PACF
ds:Earthquakes
ints:1
len_m:Max PACF
ds:Earthquakes
ints:1
len_m:None
ds:Earthquakes
ints:1
len_m:all
ds:Earthquakes
ints:10
len_m:ACF&PACF
ds:Earthquakes
ints:10
len_m:Max PACF
ds:Earthquakes
ints:10
len_m:None
ds:Earthquakes
ints:10
len_m:all
ds:Beef
ints:1
len_m:ACF&PACF
ds:Beef
ints:1
len_m:Max PACF
ds:Beef
ints:1
len_m:None
ds:Beef
ints:1
len_m:all
ds:Beef
ints:10
len_m:ACF&PACF
ds:Beef
ints:10
len_m:Max PACF
ds:Beef
ints:10
len_m:None
ds:Beef
ints:10
len_m:all
ds:DistalPhalanxOutlineAgeGroup
ints:1
len_m:ACF&PACF
ds:DistalPhalanxOutlineAgeGroup
ints:1
len_m:Max PACF
ds:DistalPhalanxOutlineAgeG

/tmp/ipykernel_71568/3028681140.py:32: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(min_bx,max_bx)


ds:Trace
ints:10
len_m:Max PACF
ds:Trace
ints:10
len_m:None
ds:Trace
ints:10
len_m:all
ds:Wine
ints:1
len_m:ACF&PACF


...

ds:MiddlePhalanxTW
ints:1
len_m:ACF&PACF
ds:MiddlePhalanxTW
ints:1
len_m:Max PACF
ds:MiddlePhalanxTW
ints:1
len_m:None
ds:MiddlePhalanxTW
ints:1
len_m:all
ds:MiddlePhalanxTW
ints:10
len_m:ACF&PACF
ds:MiddlePhalanxTW
ints:10
len_m:Max PACF
ds:MiddlePhalanxTW
ints:10
len_m:None
ds:MiddlePhalanxTW
ints:10
len_m:all
ds:MoteStrain
ints:1
len_m:ACF&PACF
ds:MoteStrain
ints:1
len_m:Max PACF
ds:MoteStrain
ints:1
len_m:None
ds:MoteStrain
ints:1
len_m:all
ds:MoteStrain
ints:10
len_m:ACF&PACF
ds:MoteStrain
ints:10
len_m:Max PACF
ds:MoteStrain
ints:10
len_m:None
ds:MoteStrain
ints:10
len_m:all
ds:Crop
ints:1
len_m:ACF&PACF
ds:Crop
ints:1
len_m:Max PACF
ds:Crop
ints:1
len_m:None
ds:Crop
ints:1
len_m:all
ds:Crop
ints:10
len_m:ACF&PACF
ds:Crop
ints:10
len_m:Max PACF
ds:Crop
ints:10
len_m:None
ds:Crop
ints:10
len_m:all
ds:Ham
ints:1
len_m:ACF&PACF
ds:Ham
ints:1
len_m:Max PACF
ds:Ham
ints:1
len_m:None
ds:Ham
ints:1
len_m:all
ds:Ham
ints:10
len_m:ACF&PACF
ds:Ham
ints:10
len_m:Max PACF
ds:Ham
ints:10
len_

/tmp/ipykernel_71568/3028681140.py:32: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(min_bx,max_bx)


ds:Fungi
ints:10
len_m:ACF&PACF
ds:Fungi
ints:10
len_m:Max PACF
ds:Fungi
ints:10
len_m:None
ds:Fungi
ints:10
len_m:all
ds:FacesUCR
ints:1
len_m:ACF&PACF


/tmp/ipykernel_71568/3028681140.py:32: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(min_bx,max_bx)


ds:FacesUCR
ints:1
len_m:Max PACF
ds:FacesUCR
ints:1
len_m:None
ds:FacesUCR
ints:1
len_m:all
ds:FacesUCR
ints:10
len_m:ACF&PACF
ds:FacesUCR
ints:10
len_m:Max PACF
ds:FacesUCR
ints:10
len_m:None
ds:FacesUCR
ints:10
len_m:all
ds:ArrowHead
ints:1
len_m:ACF&PACF
ds:ArrowHead
ints:1
len_m:Max PACF
ds:ArrowHead
ints:1
len_m:None
ds:ArrowHead
ints:1
len_m:all
ds:ArrowHead
ints:10
len_m:ACF&PACF
ds:ArrowHead
ints:10
len_m:Max PACF
ds:ArrowHead
ints:10
len_m:None
ds:ArrowHead
ints:10
len_m:all
ds:MedicalImages
ints:1
len_m:ACF&PACF
ds:MedicalImages
ints:1
len_m:Max PACF
ds:MedicalImages
ints:1
len_m:None
ds:MedicalImages
ints:1
len_m:all
ds:MedicalImages
ints:10
len_m:ACF&PACF
ds:MedicalImages
ints:10
len_m:Max PACF
ds:MedicalImages
ints:10
len_m:None
ds:MedicalImages
ints:10
len_m:all
ds:BME
ints:1
len_m:ACF&PACF
ds:BME
ints:1
len_m:Max PACF
ds:BME
ints:1
len_m:None
ds:BME
ints:1
len_m:all
ds:BME
ints:10
len_m:ACF&PACF
ds:BME
ints:10
len_m:Max PACF
ds:BME
ints:10
len_m:None
ds:BME
ints:10
len_

In [30]:
#generate hyperparameter tuning boxplots

for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
        
        
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
        df_to_cd["time"]=df_to_cd["time"]/60
        #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
        #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
        
        #print(df_to_cd.head(5))

        # Plot
        fig, ax = plt.subplots()
        
        
        #order=list(df_to_cd)
        sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
        #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
        
        max_bx=max(df_to_cd.time)
        min_bx=min(df_to_cd.time)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
        #print(df_to_cd.describe())

        # save plot
        plt.savefig('images_boxplot_time/boxplot_time'+len_m+ints+'.png')




/tmp/ipykernel_71568/4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
/tmp/ipykernel_71568/4020300904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd["time"]/60
/tmp/ipykernel_71568/4020300904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [31]:
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
        


            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.time)
            min_bx=min(df_to_cd.time)

            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]


            #df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
            #df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])
            
            #print(df_to_cd.head(5))

            # Plot
            fig, ax = plt.subplots()
            
            
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")
            #plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
            

            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
            #print(df_to_cd.describe())

            # save plot
            plt.savefig('images_boxplot_time_per_ds/boxplot_time'+len_m+ints+'_'+ds+'.png')


#### Generate CD Diagram Tunning Hyperparameter

In [32]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [33]:

df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')].classifier_name.unique()

array(['all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [34]:


from cd_function import *
#generate hyperparameter tuning cd diagrams

best_comb_by_method=[]
best=""
for len_m in df_perf.len_method.unique():
   df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')]
   draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison", fname='images_cd_diagram/cd-diagram_'+len_m+'.png')
   _, average_ranks, _ = wilcoxon_holm(df_perf=df_to_cd)
   min_rank= min(average_ranks)
   average_ranks=pd.DataFrame(average_ranks)
   best=average_ranks[average_ranks[0]==min_rank][0].index
   best_comb_by_method.append(best[0])



findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'ACF&PACF: n_random_points=10 nb_inst_per_class=1', 1: 'ACF&PACF: n_random_points=30 nb_inst_per_class=1', 2: 'ACF&PACF: n_random_points=10 nb_inst_per_class=10', 3: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10'}
[0, 1]
[2, 3]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'Max PACF: n_random_points=10 nb_inst_per_class=1', 1: 'Max PACF: n_random_points=30 nb_inst_per_class=1', 2: 'Max PACF: n_random_points=10 nb_inst_per_class=10', 3: 'Max PACF: n_random_points=30 nb_inst_per_class=10'}


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'None: n_random_points=10 nb_inst_per_class=1', 1: 'None: n_random_points=30 nb_inst_per_class=1', 2: 'None: n_random_points=10 nb_inst_per_class=10', 3: 'None: n_random_points=30 nb_inst_per_class=10'}
[0, 1]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'all: n_random_points=10 nb_inst_per_class=1', 1: 'all: n_random_points=30 nb_inst_per_class=1', 2: 'all: n_random_points=10 nb_inst_per_class=10', 3: 'all: n_random_points=30 nb_inst_per_class=10'}
[0, 1]
[2, 3]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

In [35]:
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10',
 'all: n_random_points=30 nb_inst_per_class=10']

In [36]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_to_cd=df_perf[df_perf.classifier_name.isin(best_comb_by_method)]
df_to_cd.classifier_name.unique()

array(['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=10'], dtype=object)

In [37]:
draw_cd_diagram(df_to_cd, labels=True, title="Best combination comparison", fname='images_cd_diagram/cd-diagram_best_com.png')


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'Max PACF: n_random_points=30 nb_inst_per_class=10', 1: 'None: n_random_points=30 nb_inst_per_class=10', 2: 'all: n_random_points=30 nb_inst_per_class=10', 3: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10'}
[0, 1]
[2, 3]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

In [38]:
draw_cd_diagram(df_perf, labels=True, fname='images_cd_diagram/cd-diagram_all_methods.png')


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

{0: 'None: n_random_points=10 nb_inst_per_class=1', 1: 'None: n_random_points=30 nb_inst_per_class=1', 2: 'Max PACF: n_random_points=10 nb_inst_per_class=1', 3: 'Max PACF: n_random_points=30 nb_inst_per_class=1', 4: 'None: n_random_points=10 nb_inst_per_class=10', 5: 'Max PACF: n_random_points=10 nb_inst_per_class=10', 6: 'ACF&PACF: n_random_points=10 nb_inst_per_class=1', 7: 'Max PACF: n_random_points=30 nb_inst_per_class=10', 8: 'all: n_random_points=10 nb_inst_per_class=1', 9: 'all: n_random_points=30 nb_inst_per_class=1', 10: 'ACF&PACF: n_random_points=30 nb_inst_per_class=1', 11: 'None: n_random_points=30 nb_inst_per_class=10', 12: 'all: n_random_points=10 nb_inst_per_class=10', 13: 'all: n_random_points=30 nb_inst_per_class=10', 14: 'ACF&PACF: n_random_points=10 nb_inst_per_class=10', 15: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10'}
[0, 1, 2, 3]
[6, 10, 13, 12]
[6, 10, 4, 5, 8, 9, 11, 7]
[14, 15, 8, 11]
[14, 15, 12, 13]


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

#### Focus on most accurate lenght method

##### Overall Accuracy

In [39]:
# highest accurate hyperparameters
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=10',
 'Max PACF: n_random_points=30 nb_inst_per_class=10',
 'None: n_random_points=30 nb_inst_per_class=10',
 'all: n_random_points=30 nb_inst_per_class=10']

In [40]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result
#df_result_acc=df_result_acc[df_result_acc["dataset_name"]=="Fungi"]

pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.792534,0.827653,NaN,NaN,NaN,NaN,NaN
30,NaN,0.795919,0.830512,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# create a pivot table with the variatioon of score by hyperparameter
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.019779,0.018798,NaN,NaN,NaN,NaN,NaN
30,NaN,0.020534,0.018996,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Overall time complexity

In [42]:
df_result_acc.time.sum()/(60*60*24)

11.707213821079996

In [43]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='sum')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,
Adiac,0.463301,8.145984,877.130039,7.891883,893.716162
ArrowHead,0.093649,2.876331,14.803851,0.155317,17.939097
BME,0.051486,0.459542,3.173957,0.043407,3.733848
Beef,0.200932,37.720246,60.032112,0.047557,98.026007
BeetleFly,0.138300,33.720243,16.783420,0.021030,50.677934
...,...,...,...,...,...
TwoPatterns,0.119636,0.617728,119.875390,5.683790,126.311355
UMD,0.059536,0.633915,5.205034,0.128314,6.034467
Wafer,0.106447,0.476467,51.550980,1.757995,53.900484


In [44]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.386208,3.060471,NaN,NaN,NaN,NaN,NaN
30,NaN,0.971354,8.353474,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Calculate weights time complexity

In [45]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,0.105854,0.105067
30,0.104786,0.105698


##### Finding subsequences time complexity

In [46]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,1.379207,11.641842
30,1.479735,11.928695


##### Transform Dataset time complexity

In [47]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,18.501368,167.264843
30,53.424787,483.101156


##### Classifier time complexity

In [48]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
10,3.171163,4.605198
30,3.262192,6.049180
